In [159]:
import pandas as pd
import numpy as np
import os
import time

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential, load_model
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping


import dill as pickle

In [160]:
os.getcwd()  # Get the current working directory (cwd)

'/Users/ryanmackie/Documents/DSI_coursework/Submissions/Projects/capstone-master'

In [161]:
os.chdir('/Users/ryanmackie/Documents/DSI_coursework/Submissions/Projects/capstone-master')

In [162]:
os.getcwd()

'/Users/ryanmackie/Documents/DSI_coursework/Submissions/Projects/capstone-master'

In [192]:
update_df_2 = pd.read_csv('./datasets/archive/Fake.csv')
update_df_2t = pd.read_csv('./datasets/archive/True.csv')


print(f'The dataframe has {update_df_2.shape[0]} rows and {update_df_2.shape[1]} columns.')
update_df_2.head(6)

The dataframe has 23481 rows and 4 columns.


,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017"


In [193]:
print(f'The dataframe has {update_df_2t.shape[0]} rows and {update_df_2t.shape[1]} columns.')
update_df_2t.head(6)

The dataframe has 21417 rows and 4 columns.


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"


In [194]:
update_df_2 = update_df_2[:21417]

In [195]:
print(update_df_2.shape)
print(update_df_2t.shape)

(21417, 4)
(21417, 4)


In [197]:
update_df_2 = update_df_2.drop(columns = ['title','subject', 'date'])
update_df_2t = update_df_2t.drop(columns = ['title','subject', 'date'])

In [198]:
update_df_2['Label'] = 0
update_df_2t['Label'] = 1

In [199]:
update_df_2 = pd.concat([update_df_2, update_df_2t], ignore_index = True)
update_df_2.head()

,text,Label
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [200]:
update_df_2.shape

(42834, 2)

In [201]:
update_df_2.columns = ['Story', 'Label']

In [202]:
update_df_2['Label'].value_counts(normalize = True)

1    0.5
0    0.5
Name: Label, dtype: float64

In [203]:
X = update_df_2['Story']
y = update_df_2['Label']

In [204]:
def to_words(series):
    ps = PorterStemmer()
    corpus = []
    start = time.time()
    for i in range(0, len(series)):
        letters = re.sub('[^a-zA-Z]', ' ',series[i])
        words = letters.lower().split()
        meaningful_words = [ps.stem(w) for w in words if not w in stopwords.words('english')]
        final = ' '.join(meaningful_words)
        corpus.append(final)
    end = time.time()
    print(f'Time Elapsed: {round(end - start, 2)} seconds')
    return corpus

In [205]:
vocab_size = round(20_000*1.3)

In [206]:
onehot = [one_hot(val,vocab_size)for val in to_words(X)]

Time Elapsed: 2365.47 seconds


In [1]:
# maxlen = 0
# for i in range(0, len(update_df['Story'])):
#     number = len(update_df['Story'][i].split())
#     if number > maxlen:
#         maxlen = number

padded_onehot = pad_sequences(onehot, padding = 'pre', maxlen = 2935)

In [208]:
X_final = np.array(padded_onehot)
y_final = np.array(y)

print(X_final.shape)
print(y_final.shape)

X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = .2, random_state = 33)

(42834, 2935)
(42834,)


In [213]:
model = load_model('lstm_model_2.h5')

In [214]:
model.fit(X_final, y_final, epochs=5)

Epoch 1/5
1339/1339 [==============================] - 1589s 1s/step - loss: 0.1012 - acc: 0.9700
Epoch 2/5
1339/1339 [==============================] - 1483s 1s/step - loss: 0.0310 - acc: 0.9922
Epoch 3/5
1339/1339 [==============================] - 1483s 1s/step - loss: 0.0310 - acc: 0.9904
Epoch 4/5
1339/1339 [==============================] - 1481s 1s/step - loss: 0.0207 - acc: 0.9940
Epoch 5/5
1339/1339 [==============================] - 1481s 1s/step - loss: 0.0173 - acc: 0.9950


In [215]:
model.save('lstm_model_3.h5')

In [51]:
# def model_func(layer_three_neurons=32, layer_four_neurons=16, layer_one_dropout=0.5, layer_two_dropout=0.5, layer_three_dropout=0.5, layer_four_dropout=0.5):
#     model = Sequential()
#     model.add(Embedding(vocab_size,output_dim=40,input_length=500))
#     model.add(Dropout(layer_one_dropout))
#     model.add(LSTM(256))
#     model.add(Dropout(layer_two_dropout))
#     model.add(Dense(layer_three_neurons, activation='relu'))
#     model.add(Dropout(layer_three_dropout))
#     model.add(Dense(layer_four_neurons, activation='relu'))
#     model.add(Dropout(layer_four_dropout))
#     model.add(Dense(1, activation='sigmoid')) # We can also use "linear"
#     model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])
#     return model

# # define the model with the wrapper
# nn = KerasClassifier(build_fn=model_func, epochs=10, batch_size=512)

In [52]:
# # Params grid
# params = {
#     "epochs":[10, 20],
#     "layer_three_neurons":[20, 32],
#     "layer_four_neurons":[16, 32],
#     "layer_one_dropout":[0.1, 0.3, 0.5],
#     "layer_two_dropout":[0.1, 0.3, 0.5],
#     "layer_three_dropout":[0.1, 0.3, 0.5],
#     "layer_four_dropout":[0.1, 0.3, 0.5]
# }
# gs = GridSearchCV(estimator=nn, param_grid=params, cv=2) # I'm using cv=2 for the sake of time! 
# gs.fit(X_train, y_train)
# print(gs.best_score_)
# gs.best_params_